<a href="https://colab.research.google.com/github/earendil94/SMLExam/blob/preferivolescimmie/Project_leo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STATISTICAL MACHINE LEARNING**

ARRIGHI Leonardo, BRAND Francesco, DORIGO Claudia


Dataset folder is saved in "/content/drive/My Drive/SML/SML_Project".

In [ ]:
# link colab and drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
# then follow passages

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import pandas as pd

import torchvision
from torchvision import transforms
from IPython import display

from PIL import Image
import glob

torch.manual_seed(160898)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device: {}'.format(device))

Device: cuda:0


In [ ]:
csv = '/content/drive/My Drive/SML/SML_Project/Chunk1/results.csv'
imgs = '/content/drive/My\ Drive/SML/SML_Project/Chunk1/img1_200/'

!ls {imgs} | wc

    200     200    2382


In [ ]:
# Load csv
df = pd.read_csv(csv, delimiter='|') 
print(df.shape)
print(df.columns[2], df.columns[2] == ' comment') 
df[' comment'].values[0]
df.head(6)

(158915, 3)
 comment True


,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .
5,10002456.jpg,0,Several men in hard hats are operating a gian...


In [ ]:
# Load images
# needed transformation: to tensor and normalize
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

input_images = []
for filename in glob.glob("/content/drive/My Drive/SML/SML_Project/Chunk1/img1_200/*.jpg"):
    im=Image.open(filename)
    im=transform(im) # apply transformation while loading
    input_images.append(im)


In [ ]:
for i in range(10):
  print(input_images[i].shape)
# not all images have the same shape but it shouldn't be a problem for ResNet


torch.Size([3, 500, 335])
torch.Size([3, 375, 500])
torch.Size([3, 332, 500])
torch.Size([3, 450, 450])
torch.Size([3, 338, 450])
torch.Size([3, 500, 375])
torch.Size([3, 375, 500])
torch.Size([3, 374, 500])
torch.Size([3, 338, 450])
torch.Size([3, 333, 500])


In [ ]:
import torch.optim
import torch.utils.data
from torch import nn
import torchvision.transforms as transforms
from torch.nn.utils.rnn import pack_padded_sequence


# Model parameters
#decoder_dim = 
dropout = 0.5 # suggested

# Parameters
epoch = 1000
start_epoch = 0
epochs_since_improvement = 0 # keeps track of number of epochs since there's been an improvement in validation BLEU
batch_size = 32
decoder_lr = 1e-4  # learning rate
checkpoint = None
bleu4 = 0. # bleu score -> https://www.aclweb.org/anthology/P02-1040.pdf


In [ ]:
# maybe this should be saved in a utils.py file

def adjust_learning_rate(optimizer, shrink_factor):
  for param_group in optimizer.param_groups:
    param_group['lr'] = param_group['lr'] * shrink_factor

def clip_gradient(optimizer, grad_clip):
  for i in optimizer.param_groups:
    for param in group['params']:
      if param.grad is not None:
        param.grad.data.clamp_(-grad_clip, grad_clip)



In [1]:
# Vocabulary

# Decoder
#decoder = RNN(decoder_dim, vocabolary_size, dropout = dropout)

decoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, 
                                                   decoder.parameters()),
                                     lr=decoder_lr)
decoder = decoder.to(device)

# Loss function
criterion = nn.CrossEntropyLoss().to(device)

# Epochs
for epoch in range(start_epoch, epochs):
  # Decay learning rate if there is no improvement for 8 consecutive epochs, and terminate training after 20
  if epochs_since_improvement == 20:
    break
  if epochs_since_improvement > 0 and epochs_since_improvement % 8 == 0:
    adjust_learning_rate(decoder_optimizer, 0.8)

  # Training
  train(train_loader=train_loader,
        decoder=decoder,
        criterion=criterion,
        decoder_optimizer=decoder_optimizer,
        epoch=epoch)
  
  # Validation
  #bleu4_new

  # BLEU4
  flag1 = bleu4_new > bleu4
  bleu4 = max(bleu4_new, bleu4)
  if not flag1:
    epochs_since_improvement += 1
  else:
    epochs_since_improvement = 0

NameError: ignored

In [ ]:
def train(train_loader, decoder, criterion, decoder_optimizer, epoch):

  decoder.train() # train mode

  # Batch
  for i, (image, caption, captionlen) in enumerate(train_loader):
    
    # Move to GPU, if available
    image = image.to(device)
    caption = caption.to(device)
    captionlen = captionlen.to(device)

    # Forward 
    #decoder output = decoder(image, caption, captionlen)

    # Remove timesteps that we didn't decode at, or are pads
    # https://pytorch.org/docs/master/generated/torch.nn.utils.rnn.pack_padded_sequence.html
    # https://www.tensorflow.org/tutorials/text/image_captioning
    scores, _ = pack_padded_sequence(scores, decode_lengths, batch_first=True)
    targets, _ = pack_padded_sequence(targets, decode_lengths, batch_first=True)

    # Calculate loss
    loss = criterion(scores, targets)

    # Back propagation
    decoder_optimizer.zero_grad()
    loss.backward()

    # Clip gradients
    # https://machinelearningmastery.com/how-to-avoid-exploding-gradients-in-neural-networks-with-gradient-clipping/
    if grad_clip is not None:
      clip_gradient(decoder_optimizer, grad_clip)

    # Update weights
    decoder_optimizer.step()

    
